# PRICING CALCULATIONS

In [1]:
import numpy as np
import pandas as pd

In [51]:
vol_window = 7

In [52]:
btc_price = pd.read_csv(r"..\PricingData\BitcoinHistoricalPrice.csv")[['Date', 'Close/Last']]

In [ ]:
btc_price['Return'] = btc_price['Close/Last'][::-1].pct_change()[::-1]
btc_price['Rolling Volatility'] = btc_price['Return'][::-1].rolling(vol_window).std()
btc_price

,Date,Close/Last,Return,Rolling Volatility
0,11/14/2024,87221.6,-0.031039,0.046462
1,11/13/2024,90015.6,0.042888,0.040718
2,11/12/2024,86313.8,-0.027211,0.041142
3,11/11/2024,88728.2,0.090540,0.041866
4,11/10/2024,81361.7,0.065177,0.038389
...,...,...,...,...
1791,03/12/2019,3886.0,0.005407,NaN
1792,03/11/2019,3865.1,-0.012796,NaN
1793,03/10/2019,3915.2,-0.007403,NaN
1794,03/09/2019,3944.4,0.020306,NaN


In [64]:
btc_price.describe()

,Close/Last,Return,Rolling Volatility
count,1796.000000,1795.000000,1789.000000
mean,30931.687472,0.002664,0.033313
std,20348.284242,0.047620,0.033888
min,3864.900000,-0.332928,0.001617
25%,10937.550000,-0.014881,0.019733
50%,26944.650000,0.001674,0.027882
75%,47144.450000,0.017819,0.038489
max,90015.600000,1.275501,0.481298


# TEXT CLEANING

In [67]:
pd.read_csv(r"..\RedditData\Bitcoin_new\Bitcoin_new_posts_5_years.csv")

,title,score,id,subreddit,url,num_comments,body,created,timestamp
0,Echo chamber,2,1guoe82,Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1guo...,1,Is there anyone on this sub that is not just a...,2024-11-18 21:57:46,1.731989e+09
1,Will Bitcoin move up really slow?,3,1guocyl,Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1guo...,2,"Hey,\r\n I feel like since last year when all ...",2024-11-18 21:55:42,1.731989e+09
2,Saving your company in 2024,3,1guoc0j,Bitcoin,https://i.redd.it/0xzm24ns6s1e1.jpeg,1,NaN,2024-11-18 21:54:13,1.731988e+09
3,Crypto Christmas is here. Bitcoin market drive...,0,1gunyt4,Bitcoin,https://www.cryptobull.org/blog/2024/11/18/cry...,1,Key drivers of growth for BTC and crypto marke...,2024-11-18 21:34:04,1.731987e+09
4,What is the best Bitcoin discord server?,3,1gunxeg,Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1gun...,0,I saw a previous post here asking the same que...,2024-11-18 21:31:56,1.731987e+09
...,...,...,...,...,...,...,...,...,...
983,Thought this was appropriate 🙂,3,1gpg7k3,Bitcoin,https://youtu.be/wY6insZjCfU?si=PL4tLZQDvm5xHWk1,0,NaN,2024-11-12 02:30:22,1.731400e+09
984,Why are Bitcoin buyers so loquacious about Bit...,0,1gpg69r,Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1gpg...,12,"Hello;\r\n\r\nIn the last week, I have been ab...",2024-11-12 02:27:48,1.731400e+09
985,WAKK - Forever HODL (Official Video),5,1gpg5ir,Bitcoin,https://youtu.be/FjtzrWblns0?si=9b9bQJHr_2_oYE8O,1,WE ARE NOT SELLING!! 🙌💎,2024-11-12 02:26:15,1.731400e+09
986,"Correct me if wrong, but my calculations indic...",38,1gpg4su,Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1gpg...,22,Feel free to use this information as you please.,2024-11-12 02:24:37,1.731400e+09


# COMBINE PRICING AND TEXT